# Influence of energy consumption on greenhouse gases in Turkey

This exercise uses a Bayesian Network to model the influence of energy consumption on greenhouse gases in Turkey. It has been adapted from ``pgmpy`` [Bayesian Network tutorial](https://pgmpy.org/detailed_notebooks/11.%20A%20Bayesian%20Network%20to%20model%20the%20influence%20of%20energy%20consumption%20on%20greenhouse%20gases%20in%20Italy.html) by Lorenzo Mario Amorosa, Alma Mater Studiorum – Università di Bologna (Italy).

We will analyze the effects of variations in different growth indicators on greenhouse gases via energy consumption. We are given the causal relations graph, which is the directed acyclic graph of the Bayesian Network. We will construct the network and learn its parameters from data. Then, we will use this Bayesian Network model to obtain probabilistic results about greenhouse gases given some evidence.

This exercise uses [Bayesian Network model](https://pgmpy.org/models/bayesiannetwork.html) in ``pgmpy``.

## Constructing the network

We will use the following variables in our model construction with the hypothesis that growth rates of population, urbanization, and gross domestic product (GDP) affect energy use in a country, and energy use affects the emission of greenhouse gases.

* POP = Population growth (annual %)
* URB = Urban population growth (annual %)
* GDP = GDP per capita growth (annual %)
* EC = Energy use (kg of oil equivalent per capita) - [annual growth %]
* FFEC = Fossil fuel energy consumption (% of total) - [annual growth %]
* REC = Renewable energy consumption (% of total final energy consumption) - [annual growth %]
* EI = Energy imports, net (% of energy use) - [annual growth %]
* CO2 = CO2 emissions (metric tons per capita) - [annual growth %]
* CH4 = Methane emissions in energy sector (thousand metric tons of CO2 equivalent) - [annual growth %]
* N2O = Nitrous oxide emissions in energy sector (thousand metric tons of CO2 equivalent) - [annual growth %]

The causal relations between the variables are shown in the below graph.

<img src="images/BN.png" alt="Bayesian Network" width="400">

Please construct the graph using ``BayesianNetwork`` model in ``pgmpy``. (You can check the API in here: https://pgmpy.org/models/bayesiannetwork.html)

In [ ]:
from pgmpy.models import BayesianNetwork

# please provide the directed edges as a list of tuples
# be careful about edge directions!
model = BayesianNetwork([('POP', 'EC'), ('URB','EC' ), ('GDP','EC'),('EC','FFEC'),('EC','REC'),('EC','EI'),('FFEC','CO2'),('FFEC','CH4'),('FFEC','N20'),('REC','CH4'),('REC','N20'),('REC','CO2')])


## Dataset

The data has been collected from [the databank of The World Bank](https://databank.worldbank.org). We will compute conditional probability tables (CPTs) of the Bayesian Network using this data. Before moving into CPT calculation, we need to prepare the data. Some variables in the dataset are not in annual growth format, we will convert them to annual growth percentages and append " - [annual growth %]" to variable names.

### Raw data

In [ ]:
from pandas import read_csv, DataFrame
import numpy as np

def annual_growth(row, years):
    min_year = years["min"]
    max_year = years["max"]
    row["Series Name"] = row["Series Name"] + " - [annual growth %]"
    for year in range(max_year, min_year, -1):
        if not np.isnan(row[str(year)]) and not np.isnan(row[str(year - 1)]):
            row[str(year)] = 100 * (float(row[str(year)]) - float(row[str(year - 1)])) / abs(float(row[str(year - 1)]))
        else:
            row[str(year)] = np.nan
    row[str(min_year)] = np.nan
    return row

years = {"min" : 1960, "max" : 2021}
df_raw = read_csv("raw_data.csv")
df_raw_growth = DataFrame(data=[row if "growth" in row["Series Name"] else annual_growth(row, years) for index, row in df_raw.iterrows()])
print("There are " + str(df_raw_growth.shape[0]) + " indicators in the dataframe.")
df_raw_growth.head(11)

### Data cleaning

This step drops the unused columns in the data.

In [ ]:
nodes = ['POP', 'URB', 'GDP', 'EC', 'FFEC', 'REC', 'EI', 'CO2', 'CH4', 'N2O']
df_growth = df_raw_growth.transpose().iloc[4:]
df_growth.columns = nodes
df_growth.head(10)

### Data discretization

We will also discretize the values of variables by using binning. Low, medium, and large are binned into 'A', 'B', and 'C', respectively.

*Please note that there are some missing values, but ``pgmpy`` can handle missing data!*

In [ ]:
TIERS_NUM = 3

def boundary_str(start, end, tier):
#     return f'{tier}: {start:+0,.2f} to {end:+0,.2f}'
    return f'{tier}'

def print_boundaries(label, boundaries):
    print('{} - A: {:.2f} to {:.2f}, B: {:.2f} to {:.2f}, C: {:.2f} to {:.2f}'.format(label,boundaries[0][0], boundaries[0][1],boundaries[1][0], boundaries[1][1],boundaries[2][0], boundaries[2][1]))

def relabel(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return boundary_str(boundaries[0][0], boundaries[0][1], tier='A')
    elif v >= boundaries[1][0] and v <= boundaries[1][1]:
        return boundary_str(boundaries[1][0], boundaries[1][1], tier='B')
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return boundary_str(boundaries[2][0], boundaries[2][1], tier='C')
    else:
        return np.nan

def get_boundaries(tiers):
    prev_tier = tiers[0]
    boundaries = [(prev_tier[0], prev_tier[prev_tier.shape[0] - 1])]
    for index, tier in enumerate(tiers):
        if index is not 0:
            boundaries.append((prev_tier[prev_tier.shape[0] - 1], tier[tier.shape[0] - 1]))
            prev_tier = tier
    return boundaries

new_columns = {}
for i, content in enumerate(df_growth.items()):
    (label, series) = content
    values = np.sort(np.array([x for x in series.tolist() if not np.isnan(x)] , dtype=float))
    if values.shape[0] < TIERS_NUM:
        print(f'Error: there are not enough data for label {label}')
        break
    boundaries = get_boundaries(tiers=np.array_split(values, TIERS_NUM))
    print_boundaries(label, boundaries)
    new_columns[label] = [relabel(value, boundaries) for value in series.tolist()]

df = DataFrame(data=new_columns)
df.columns = nodes
df.index = range(years["min"], years["max"] + 1)
df.head(10)

## Learning network parameters

In this part, we will learn CPTs of the Bayesian Network. We will use ``model.fit()`` with ``BayesianEstimator`` and ``MaximumLikelihoodEstimator`` from ``pgmpy.estimators``. We can specify if we will learn CPTs using only complete data (``complete_samples_only=False``) or using all data with some missing values (``complete_samples_only=True``).

*Please note that ``BayesianEstimator`` requires a prior type to use for the model parameters. We will use ``prior_type="BDeu"`` with ``equivalent_sample_size=10`` ([see details in here](https://pgmpy.org/param_estimator/bayesian_est.html)).*

In [ ]:
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator
from IPython.core.display import display, HTML

# disable text wrapping in output cell
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

model.cpds = []
model.fit(data=df,
          estimator=BayesianEstimator,
          prior_type="BDeu",
          equivalent_sample_size=10,
          complete_samples_only=False)

print(f'Check model: {model.check_model()}\n')
for cpd in model.get_cpds():
    print(f'CPT of {cpd.variable}:')
    print(cpd,'\n')

In [ ]:
# POP(B)
print('POP(B): {:.3f}'.format(model.get_cpds("POP").values[1]))

## Network analysis

### Some definitions

**In a Bayesian Network:**

* A variable is conditionally independent of its **non-descendants**, given its parents
* A variable is conditionally independent of all other nodes in the network, given its parents, children, and children's parents - that is given its **Markov blanket**.
* Given three nodes $X$, $Y$, and $Z$, when influence can flow from $X$ to $Y$ via $Z$, we say that the **trail** $X \rightleftharpoons Z \rightleftharpoons Y$ is **active**. There are for possible active two-edge trails:
    * **Causal trail** $X \rightarrow Z \rightarrow Y$: active if and only if $Z$ is not observed
    * **Evidential trail** $X \leftarrow Z \leftarrow Y$: active if and only if $Z$ is not observed
    * **Common cause** $X \leftarrow Z \rightarrow Y$: active if and only if $Z$ is not observed
    * **Common effect** $X \rightarrow Z \leftarrow Y$: active if and only if either $Z$ or one of $Z$'s descendants is observed

Let's consider a longer trail $X_1 \rightleftharpoons \cdots \rightleftharpoons X_n$. Intiutively, for influence to "flow" from $X_1$ to $X_n$, it needs to flow through every single node on the trail. In other words, $X_1$ can influence $X_n$ if every two-edge trail $X_{i-1} \rightleftharpoons X_i \rightleftharpoons X_{i+1}$ along the trail allows influence to flow.

Let $\mathcal{G}$ be a Bayesian Network structure and $\mathbf{X}$, $\mathbf{Y}$, $\mathbf{Z}$ be three sets of nodes in $\mathcal{G}$. We say that $\mathbf{X}$ and $\mathbf{Y}$ are **d-seperated** given $\mathbf{Z}$, denoted $\text{d-sep}_{\mathcal{G}}(\mathbf{X};\mathbf{Y}|\mathbf{Z})$, if there is no active trail between any node $X \in \mathbf{X}$ and $Y \in \mathbf{Y}$ given $\mathbf{Z}$.

### Task
This section analyzes the network in terms of independencies.

In [ ]:
print(f'There can be made {len(model.get_independencies().get_assertions())}',
      'valid independence assertions with respect to the all possible given evidence.')
print('For instance, any node in the network is independent of its non-descendents given its parents (local semantics):\n',
      f'\n{model.local_independencies(nodes)}\n')

def active_trails_of(query, evidence):
    active = model.active_trail_nodes(query, observed=evidence).get(query)
    active.remove(query)
    if active:
        if evidence:
            print(f'Active trails between \'{query}\' and {active} given the evidence {set(evidence)}.')
        else:
            print(f'Active trails between \'{query}\' and {active} given no evidence.')
    else:
        print(f'No active trails for \'{query}\' given the evidence {set(evidence)}.')

def markov_blanket_of(node):
    print(f'Markov blanket of \'{node}\' is {set(model.get_markov_blanket(node))}')

active_trails_of(query='FFEC', evidence=[])
active_trails_of(query='FFEC', evidence=['EC'])
active_trails_of(query='FFEC', evidence=['EC', 'CO2'])
active_trails_of(query='EI', evidence=['EC'])
active_trails_of(query='CH4', evidence=['EC', 'FFEC'])
print()
markov_blanket_of(node='POP')
markov_blanket_of(node='EC')
markov_blanket_of(node='REC')
markov_blanket_of(node='CH4')

### Some queries on independence assertions

In [ ]:
from pgmpy.independencies.Independencies import IndependenceAssertion

def check_assertion(model, independent, from_variables, evidence):
    assertion = IndependenceAssertion(independent, from_variables, evidence)
    result = False
    for a in model.get_independencies().get_assertions():
        if frozenset(assertion.event1) == a.event1 and assertion.event2 <= a.event2 and frozenset(assertion.event3) == a.event3:
            result = True
            break
    print(f'{assertion}: {result}')

In [ ]:
check_assertion(model, independent=['EI'], from_variables=['N2O'], evidence=['EC'])

## Inferences

In this part, we will make inferences using variable elimination over the Bayesian Network. We will use ``VariableElimination`` in ``pgmpy``'s inference module ([see details in here](https://pgmpy.org/exact_infer/ve.html))

In [ ]:
from pgmpy.inference import VariableElimination
import time

def query_report(infer, variables, evidence=None, elimination_order="MinFill", show_progress=False, desc=""):
    if desc:
        print(desc)
    start_time = time.time()
    print(infer.query(variables=variables,
                      evidence=evidence,
                      elimination_order=elimination_order,
                      show_progress=show_progress))
    print(f'--- Query executed in {time.time() - start_time:0,.4f} seconds ---\n')

infer = VariableElimination(model)

In [ ]:
var = ['CO2']
ev = {'EC': 'A'}
query_report(infer, variables=var, evidence=ev, desc=f'Probability query of {var} with evidence {ev}:')